# Aula 09 - Fluxo Ótico

Exemplo adaptado do link: https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html

Referências: 
    1. Szeilisk - Cap 8.4
    1. Artigo Bouguet - Pyramidal Implementation of the Lucas Kanade Feature Tracker


In [1]:
%reset -f
%matplotlib inline

#!pip install opencv-python

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv.__version__)

3.4.2


___

1. Realizar o download das imagens no Blackboard e verificar se o notebook consegue abri-las:

In [2]:
# Abre uma imagem em gray scale
img = cv.imread('./Imagens/frame_0050.jpg', 0)

# Exibe a imagem
cv.imshow('image',img)
cv.waitKey(0)
cv.destroyAllWindows()

___

2. Detectar alguns pontos interessantes nela usando o método visto na aula passada.

In [3]:
# Abre a imagem
img = cv.imread('./Imagens/frame_0050.jpg')

# Converte para Grayscale
bw = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Parametriza a funcao do OpenCV
params = dict( maxCorners = 100,
               qualityLevel = 0.3,
               minDistance = 7,
               blockSize = 7 )

# Funcao que retorna uma lista de pontos
pts = cv.goodFeaturesToTrack(bw, mask = None, **params)

## Plota os pontos na imagem:

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

# Insere uma marcação em cada ponto:
for i,pt in enumerate(pts):
        a,b = pt.ravel()
        img = cv.circle(img,(a,b),5,color[i].tolist(),-1)

# Exibe a imagem
cv.imshow('image',img)
cv.waitKey(0)
cv.destroyAllWindows()

___

3. Agora vamos realizar o tracking dos pontos usando fluxo ótico. 

  * Ver o artigo do Bouguet no Blackboard para mais detalhes.
  * Fonte das imagens: http://www.cvg.reading.ac.uk/PETS2009/a.html

In [2]:
# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

# Detecta os pontos no primeiro frame. Será usado como base na próxima imagem.
previous = cv.imread('./Imagens/frame_0050.jpg')
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

# Abre o restante das imagens
for i in range(51,71):

    actual = cv.imread('./Imagens/frame_{:04d}.jpg'.format(i))
    
    actual_gray = cv.cvtColor(actual, cv.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    img = cv.add(actual, mask)
    
    cv.imshow('frame', img)
    cv.waitKey(0)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
cv.destroyAllWindows()

## Projeto 2-1: Estabilização de imagens

**Motivação**: https://www.youtube.com/watch?v=4vt7bGEen2s

Agora que você consegue capturar o fluxo ótico entre duas imagens consecultivas, vamos utilizá-lo de modo inverso: como seria uma forma de compensar eventuais oscilações na câmera?

Projete um programa que captura as imagens da webcam e realiza a estabilização da imagem. Você notará que se utilizar o programa acima, a estabilização será parcial e falha. Por que?

Você deve construir um Jupyter Notebook que utiliza o Dense Optical Flow para corrigir o problema acima. O notebook deve conter comentários acerca da solução usada.

O programa base para uso da webcam:

```python
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)
 
while(1):
    ret, frame = captura.read()
    cv2.imshow("Video", frame)
   
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.5)
 
captura.release()
cv2.destroyAllWindows()
```

### Entrega: 12/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da Aula 09.
    C. Utiliza o Dense Optical Flow de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

In [13]:
%reset -f
%matplotlib inline

#!pip install opencv-python

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv2.__version__)

3.4.2


In [14]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)
 
while(1):
    ret, frame = captura.read()
    cv2.imshow("Video", frame)
   
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
 
captura.release()
cv2.destroyAllWindows()

In [7]:
%reset -f

# https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(0)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    print("Good new:")
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
#         mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv.destroyAllWindows()
cap.release()

In [3]:
%reset -f
import cv2 as cv
import numpy as np
cap = cv.VideoCapture(0)
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',next)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',frame2)
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv.destroyAllWindows()